In [17]:
%%configure -f
{"driverMemory": "4g", "executorMemory":"8960m"}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1646196661453_0015,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1646196661453_0015,pyspark,idle,Link,Link,None,✔


In [18]:
# Packages to download the corpus and pandas
import urllib.request
#import pandas as pd
# This block is for sparknlp stuff
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
# This block is for pyspark stuff
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import BucketedRandomProjectionLSH, BucketedRandomProjectionLSHModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
hashesDF = spark.read.parquet("hdfs:///user/diego/output").persist()
brp_model = BucketedRandomProjectionLSHModel.load("hdfs:///user/diego/brp_model.model")
model_use = PipelineModel.load("hdfs:///user/diego/model_use.model")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
def find_close_sentences(query, emb_model, brp_model, hashesDF, k):
  key = get_key(query, emb_model)
  resultsDF = brp_model.approxNearestNeighbors(hashesDF, key, k)
  return resultsDF

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
def get_key(query, model):
  queryDF = spark.createDataFrame([[query]]).toDF("text")
  queryDF = model.transform(queryDF)
  queryDF = queryDF.select(
    F.explode(
      F.arrays_zip(
        F.col("sentence.result"),
        F.col("use_embeddings.embeddings")
      )
    ).alias("zip")
  ).select(
    F.col("zip")['0'].alias("sentence"),
    myudf(F.col("zip")['1']).alias("embeddings")
  )  

  key = queryDF.select("embeddings").take(1)[0].embeddings
  return key

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
def avg_vectors(bert_vectors):
  
  length = len(bert_vectors[0]["embeddings"])
  avg_vec = [0] * length
  
  for vec in bert_vectors:
    for i, x in enumerate(vec["embeddings"]):
      avg_vec[i] += x
    avg_vec[i] = avg_vec[i] / length
  return avg_vec

import pyspark.sql.types as T

avg_vectors_udf = F.udf(avg_vectors, T.ArrayType(T.DoubleType()))

myudf = F.udf(lambda vs: Vectors.dense(vs), VectorUDT())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
find_close_sentences("Hadoop", model_use, brp_model, hashesDF, 5).select(F.col("title"),F.col("distCol"),F.col("sentence")).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                                                                                                                                                      |distCol           |sentence                                                                          